### Extracting classes that are relevant to deforestation

The original MODIS classes that are relevant to the project are as follows:

- Forest classes = 1–5 (IGBP scheme)
- Agriculture classes = 12, 14
- Pasture/shrubland = 6–10

With this, I aim to simplify and keep only the relevant classes for analysis.

So, this is being done in such a way that I have 4 major land-use categories:

0 - Other land, which includes wetlands, water, urban areas, barren land, etc.

1 - Forest, which includes evergreen and deciduous forest types. These will be valuable in tracking deforestation.

2 - Agriculture, which includes cropland/natural mosaic. These would be used to track agricultural expansion.

3 - Pasture/shrub/savanna, which includes shrublands, savannas. Although not sure yet, these would perhaps be useful to understand forest to pasture transitions.

The output of this code will be:
- *_forest.tif - binary forest mask
- *_agri.tif - binary agricultural mask
- *_pasture.tif - binary pasture mask
- *_multiclass.tif - combined raster for land-use change analysis

Each of these rasters will be available for each year of analysis, i.e., 2001-2024

In [1]:
# importing the necessary libraries
import rasterio
import os
import numpy as np

In [2]:
# verifying the projection of the tif files
with rasterio.open("/shared_space/BrazilSPEI/MODIS/Masked_Data/MCD12Q1_Maranhao_2001.tif") as src:
    print(src.crs)


EPSG:4326


In [3]:
input_dir = "/shared_space/BrazilSPEI/MODIS/Data"
output_dir = "/shared_space/BrazilSPEI/MODIS/Reclass_Data"
os.makedirs(output_dir, exist_ok = True)

In [4]:
forest_classes = [1, 2, 3, 4, 5]
agri_classes = [12, 14]
pasture_classes = list(range(6, 10)) # I don't know if I will use this but would be interesting to see if there's been a rise in these lands

In [8]:
for fname in sorted(os.listdir(input_dir)):
    if not fname.endswith(".tif"):
        continue
        
    input_path = os.path.join(input_dir, fname)
    with rasterio.open(input_path) as src:
        arr = src.read(1) # opening a single band
        meta = src.meta.copy()
        
    # forest mask
    forest_mask = np.isin(arr, forest_classes).astype("uint8")
    
    # agricultural mask
    agri_mask = np.isin(arr, agri_classes).astype("uint8")
    
    # pasture mask
    pasture_mask = np.isin(arr, pasture_classes).astype("uint8")
    
    # creating a multi-class map
    # 0 - other, 1 - forest, 2 - agriculture, 3 - pasture/shrub/savanna
    multiclass = np.zeros_like(arr, dtype = "uint8")
    multiclass[np.isin(arr, forest_classes)] = 1
    multiclass[np.isin(arr, agri_classes)] = 2
    multiclass[np.isin(arr, pasture_classes)] = 3
    
    meta.update(dtype = "uint8", count = 1)
    
    # saving the outputs
    out_forest = os.path.join(output_dir, fname.replace(".tif", "_forest.tif"))
    out_pasture = os.path.join(output_dir, fname.replace(".tif", "_pasture.tif"))
    out_agri = os.path.join(output_dir, fname.replace(".tif", "_agri.tif"))
    out_multi = os.path.join(output_dir, fname.replace(".tif", "_multiclass.tif"))
    
    with rasterio.open(out_forest, "w", **meta) as dst:
        dst.write(forest_mask, 1)
        
    with rasterio.open(out_pasture, "w", **meta) as dst:
        dst.write(pasture_mask, 1)
        
    with rasterio.open(out_agri, "w", **meta) as dst:
        dst.write(agri_mask, 1)
    
    with rasterio.open(out_multi, "w", **meta) as dst:
        dst.write(multiclass, 1)
        
    print("Processed:", fname)

Processed: MCD12Q1_Maranhao_2001.tif
Processed: MCD12Q1_Maranhao_2002.tif
Processed: MCD12Q1_Maranhao_2003.tif
Processed: MCD12Q1_Maranhao_2004.tif
Processed: MCD12Q1_Maranhao_2005.tif
Processed: MCD12Q1_Maranhao_2006.tif
Processed: MCD12Q1_Maranhao_2007.tif
Processed: MCD12Q1_Maranhao_2008.tif
Processed: MCD12Q1_Maranhao_2009.tif
Processed: MCD12Q1_Maranhao_2010.tif
Processed: MCD12Q1_Maranhao_2011.tif
Processed: MCD12Q1_Maranhao_2012.tif
Processed: MCD12Q1_Maranhao_2013.tif
Processed: MCD12Q1_Maranhao_2014.tif
Processed: MCD12Q1_Maranhao_2015.tif
Processed: MCD12Q1_Maranhao_2016.tif
Processed: MCD12Q1_Maranhao_2017.tif
Processed: MCD12Q1_Maranhao_2018.tif
Processed: MCD12Q1_Maranhao_2019.tif
Processed: MCD12Q1_Maranhao_2020.tif
Processed: MCD12Q1_Maranhao_2021.tif
Processed: MCD12Q1_Maranhao_2022.tif
Processed: MCD12Q1_Maranhao_2023.tif
Processed: MCD12Q1_Maranhao_2024.tif


In [7]:
path = '/shared_space/BrazilSPEI/MODIS/Data/MCD12Q1_Maranhao_2001.tif'

with rasterio.open(path) as src:
    arr = src.read(1)

print("Unique values:", np.unique(arr)[:50])

Unique values: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 16 17]
